# Event-based Constellation Example

This notebook illustrates how to run many PASEOS instances in parellel to model a constellation in low-Earth orbit (LEO). 

First, let's import all necessary modules

In addition to PASEOS's requirements this notebook uses seaborn and pandas. `conda install seaborn` or `pip install seaborn` will get it. (analogously for pandas)

In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys 
sys.path.append("..")
sys.path.append("../..")

import warnings
warnings.filterwarnings("ignore")

import pykep as pk
import numpy as np
import paseos
from paseos.actors.spacecraft_actor import SpacecraftActor
from paseos.actors.actor_builder import ActorBuilder
from paseos.utils.link_budget_calc import calc_dist_and_alt_angle
from paseos.communication.device_type import DeviceType
from paseos.communication.link_model import LinkModel

# paseos.set_log_level("WARNING") # use info / debug if you want more insight


### Constellation setup

In addition to the code in this notebook, we use a simple function that automatically computes the orbits of a [Walker Constellation](https://en.wikipedia.org/wiki/Satellite_constellation#Walker_Constellation) for us. It allows us to only specify a few parameters to get started.

For illustrative purposes our constellation will consist of 16 satellites in LEO and one communication satellite in GEO. Feel free to change the numbers to try out different setups.

In [ ]:
from utils.get_constellation import get_constellation
altitude = 550 * 1000 # constellation altitude above the Earth's ground [m]
inclination = 10.0 # inclination of each orbital plane
nPlanes = 1 # the number of orbital planes (see linked wiki article)
nSats = 1 # the number of satellites per orbital plane
t0 = pk.epoch_from_string("2023-Jan-04 20:00:00") # the starting date of our simulation

# Compute orbits of LEO satellites
planet_list,sats_pos_and_v,orbital_period = get_constellation(altitude,inclination,nSats,nPlanes,t0)

# Compute orbit of GEO communications satellite
altitude_geo = 35786*1000
inclination_geo,n_Planes_geo,nSats_geo = 0,1,1
comms_sat,comm_sat_pos_and_v,_ = get_constellation(altitude_geo,inclination_geo,n_Planes_geo,nSats_geo,t0)

Let's briefly plot our satellite orbits for illustrative purposes. We use [pykep's plotting features](https://esa.github.io/pykep/documentation/orbitplots.html) for that.

Note that angles aren't to scale in pykep plots.

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d')
colors = sns.color_palette("Paired")
for i in range (nPlanes*nSats):
    color_idx = i // nPlanes
    pk.orbit_plots.plot_planet(planet_list[i],axes=ax,s=64,color=colors[color_idx])
# pk.orbit_plots.plot_planet(comms_sat[0],axes=ax,s=64)

### PASEOS Setup

Now we will create a PASEOS instance for each LEO satellite. Further, we add a communication device, power device and a thermal model.

Pleae have a look at the [readme sections on physical models](https://github.com/aidotse/PASEOS#physical-models) for additional details.

In [ ]:
paseos_instances = [] # this will store paseos instances
earth = pk.planet.jpl_lp("earth") # define our central body

comms_instances = []

pos,v = comm_sat_pos_and_v[0]
comms_sat:SpacecraftActor = ActorBuilder.get_actor_scaffold(name="comms_1",actor_type=SpacecraftActor, epoch=t0)
ActorBuilder.set_orbit(actor=comms_sat,position=pos,velocity=v,epoch=t0,central_body=earth)

# Optical receiver input based on 
# https://www.eoportal.org/satellite-missions/edrs#lct-laser-communication-terminal
optical_receiver_name = "optical_receiver_1"
ActorBuilder.add_comm_device(actor=comms_sat,device_name=optical_receiver_name, line_losses=4.1, antenna_diameter=135E-3, device_type=DeviceType.OPTICAL_RECEIVER)    

instance = paseos.init_sim(local_actor=comms_sat)
comms_instances.append(instance)

for idx,sat_pos_v in enumerate(sats_pos_and_v):
    pos,v = sat_pos_v
    sat_actor:SpacecraftActor = ActorBuilder.get_actor_scaffold(name="Sat_"+str(idx),
                                                actor_type=SpacecraftActor,
                                                epoch=t0)
    ActorBuilder.set_orbit(actor=sat_actor,position=pos,velocity=v,epoch=t0,central_body=earth)
    optical_transmitter_name = "sat_optical_transmitter_1"

    # Optical transmitter input based on Giggenbach, D., Knopp, M. T., & Fuchs, C. (2023).
    # Link budget calculation in optical LEO satellite downlinks with on/off‐keying and
    # large signal divergence: A simplified methodology. International Journal of Satellite
    # Communications and Networking.
    ActorBuilder.add_comm_device(actor=sat_actor,device_name=optical_transmitter_name, input_power=1, power_efficiency=1, antenna_efficiency=1, line_losses=1, point_losses=3, fwhm=1E-3, device_type=DeviceType.OPTICAL_TRANSMITTER)
    
    ActorBuilder.set_power_devices(actor=sat_actor,battery_level_in_Ws=10000+np.random.rand()*90000,
                                   max_battery_level_in_Ws=100000,charging_rate_in_W=50)
    ActorBuilder.set_thermal_model(
            actor=sat_actor,
            actor_mass=50.0,
            actor_initial_temperature_in_K=273.15,
            actor_sun_absorptance=1.0,
            actor_infrared_absorptance=1.0,
            actor_sun_facing_area=2.0,
            actor_central_body_facing_area=2.0,
            actor_emissive_area=4.0,
            actor_thermal_capacity=1000
    )
    optical_link_name = "optical_link_1"
    ActorBuilder.add_comm_link(sat_actor, optical_transmitter_name, comms_sat, optical_receiver_name, optical_link_name)
    
    instance = paseos.init_sim(local_actor=sat_actor)
    paseos_instances.append(instance)

We can use PASEOS internal plotting to have a look at our setup now. As of writing the plotting isn't ideal for larger constellations ¯\_(ツ)_/¯

In [ ]:
paseos_instances[0].empty_known_actors()
for instance in paseos_instances[1:]:
    paseos_instances[0].add_known_actor(instance.local_actor)
plotter = paseos.plot(paseos_instances[0], paseos.PlotType.SpacePlot)

Now, we define some operational constraints for our constellation. Let's say our LEO satellites have two tasks:

* Processing - Consumes 100W and can only be performed at < 56.85° C and if our battery is at least 20% charged.
* Standby - What we do if the above constraint is violated (costs 2W)

To this end, we define a function that returns `True` if the constraint is met.

In [ ]:
def get_consumption_and_activity(actor):
    """Determine power consumption and activity for an actor."""
    if operational_constraint(actor):
        value = 100
        action = "Processing"
        transmitters = actor.get_all_transmitters()
        for transmitterName in transmitters:
            transmitter = actor.get_transmitter(transmitterName)
            if transmitter.active:
                value += transmitter.input_power
                action += "Communicating"

        # Power consumption for processing is 100W
        return value,action
    else: 
        # Power consumption in standby is 2W
        return 2.0, "Standby"

def operational_constraint(actor):
    """Determine if constraint is met for an actor"""
    if (actor.state_of_charge > 0.2 
        and actor.temperature_in_K < 330):
        return True
    else:
        return False

### Running the simulation

Now, we are ready to run the main simulation. We will simulate the operations of this constellation for 8 hours. Every 600 seconds (or once the constraint is no longer valid) we will redecide whether a satellite is going to "charge" or "process".

Whenever a satellite starts violating the operational constraint, it will stop the activity and continue to standby. This is marked by an `INFO` output from PASEOS.

If the activity is interrupted due to overheating or lack of battery charge, we continue charging in standby for the remaining time of the 600 second window.

In [ ]:
simulation_time = 6.0 * 3600 # eight hours in seconds
t = 0 # starting time in seconds
timestep = 1200 # how often we decide what each sat should do [s]

# Run until end of simulation
while t <= simulation_time: 
    N_standby = 0 # track in standby
    N_processing = 0 # track processors
    N_interruped = 0 # track which switched due to constraint
    # For each satellite, we perform the following steps
    for instance in paseos_instances:
        local_t = instance.local_actor.local_time
        # Update known actors, i.e. for each sat if they can
        # see the ground station and the comms satellite
        for link in instance.local_actor.communication_links:
            distance, elevation_angle = calc_dist_and_alt_angle(instance.local_actor, link.receiver_actor, local_t)
            if instance.local_actor.is_in_line_of_sight(link.receiver_actor, epoch=local_t):
                link.set_line_of_sight(True)
                bitrate = link.get_bitrate(distance, elevation_angle)
                link.set_bitrate(bitrate)
            else:
                link.set_bitrate(0)
                link.set_line_of_sight(False)
            link.set_distance(distance)
            link.set_elevation_angle(elevation_angle)
                
        instance.empty_known_actors()

        # Determine whether satellite is processing or in standby
        power_consumption, activity = get_consumption_and_activity(instance.local_actor)
        if "Processing" in activity:
            N_processing += 1
            eval_constraint = lambda: operational_constraint(instance.local_actor)
        else:
            N_standby += 1
            eval_constraint = None
        
        # This call is only necessary to display running activity 
        # in PASEOS internal monitoring (which we use below)
        instance.local_actor._current_activity = activity

        # Advance the simulation state of this satellite
        # Note how we pass the "eval_constraint" to tell paseos to evaluate if the constraint
        # for running the "Processing" is still satisfied
        time_remaining_to_advance = instance.advance_time(
            time_to_advance=timestep,
            current_power_consumption_in_W=power_consumption,
            constraint_function=eval_constraint
        )

        # If activity was interrupted by constraint violation, proceed in standby for remainder
        if time_remaining_to_advance > 0:
            N_interruped += 1
            instance.advance_time(
            time_to_advance=time_remaining_to_advance,
            current_power_consumption_in_W=2.0,
            constraint_function=None
        )
    # Advance the comm satellite and ground station
    for instance in comms_instances:
        instance.advance_time(time_to_advance=timestep,current_power_consumption_in_W=0.0)
            
    print(f"Time: {t}s - # of Processing = {N_processing} ({N_interruped} interrupted), # of Standby = {N_standby}")
    t += timestep

### Plotting and Analysis

Alright! Now, let's have a look at what actually happened during our simulation.

For this, we will make use of [PASEOS' monitor](https://github.com/aidotse/PASEOS#monitor).

The below code uses it to extract a few quantities for all our sats and plots them over the course of the simulation for each satellite (each color is one).

This will give us a good impression of whether each satellite's operations are sensible.

We will also accumulate all the data into a [pandas dataframe](https://pandas.pydata.org/docs/user_guide/10min.html) for convenience.

In [ ]:
from communication_example_utils import get_known_actor_comms_status, get_bitrate_status, get_line_of_sight_status, get_distance_status, get_elevation_angle_status

# Quantities we want to track
quantities = ["temperature","state_of_charge","current_activity","known_actors","is_in_eclipse", "bitrate", "line_of_sight", "distance", "elevation_angle"]

# Setup data frame to collect all data
df = pd.DataFrame(columns=("Time","ID"))
for idx,item in enumerate(quantities):
    names = []

    # Dataframe for this quantity
    small_df = pd.DataFrame(columns=("Time","ID"))
    
    plt.figure(figsize=(8, 2),dpi=150)

    # Get data from all satellites and plot it
    for instance in paseos_instances:

        # Get time of each data point
        timesteps = instance.monitor["timesteps"]

        # Get data
        communication_histories = []

        if item == "known_actors":
            values = get_known_actor_comms_status(instance.monitor[item])
        elif item == "bitrate":
            for link in instance.local_actor.communication_links:
                communication_histories.append(get_bitrate_status(link))
        elif item == "line_of_sight":
            for link in instance.local_actor.communication_links:
                communication_histories.append(get_line_of_sight_status(link))
        elif item == "distance":
            for link in instance.local_actor.communication_links:
                communication_histories.append(get_distance_status(link))
        elif item == "elevation_angle":
            for link in instance.local_actor.communication_links:
                communication_histories.append(get_elevation_angle_status(link))
        else:
            values = instance.monitor[item]
        names.append(instance.local_actor.name)

        if item == "bitrate" or item == "line_of_sight" or item == "distance" or item == "elevation_angle":
            # Collect data from this sat into big dataframe
            for history in communication_histories:
                plt.figure(figsize=(8, 2),dpi=150)
                smaller_df = pd.DataFrame({"Time": timesteps,"ID": len(timesteps)*[instance.local_actor.name],item: history})
                small_df = pd.concat([small_df,smaller_df])
                if item == "line_of_sight":
                    smaller_df[item] = smaller_df[item].astype("boolean")
                plt.plot(timesteps,history)
                plt.xlabel("Time [s]")
                plt.ylabel(item)
        else:
            smaller_df = pd.DataFrame({"Time": timesteps,"ID": len(timesteps)*[instance.local_actor.name],item: values})
            if item == "is_in_eclipse": #pandas things...
                smaller_df["is_in_eclipse"] = smaller_df["is_in_eclipse"].astype("boolean")
            small_df = pd.concat([small_df,smaller_df])
        
            # Plot it :)
            plt.plot(timesteps,values)
            plt.xlabel("Time [s]")
            plt.ylabel(item.replace("_", " "))

    # Add a legend showing which satellite is which
    # plt.legend(
    #         names,
    #         fontsize = 8,
    #         bbox_to_anchor=(0.5, 1.4),
    #         ncol=10,
    #         loc="upper center",
    # )
    
    df = df.merge(small_df,how="right")